In [1]:
# Problem Statement
# Dorian Auto is considering manufacturing three types of autos: compact, midsize, and large. 
# The resources required for, and the profits yielded by, each type of car are shown in Table 8. 
# Currently, 6,000 tons of steel and 60,000 hours of labor are available. For production of a type 
# of car to be economically feasible, at least 1,000 cars of that type must be produced. 

# Formulate an IP to maximize Dorian’s profit.

![](Dorian-Auto.png)

# Translating a Decision Problem to an Optimization Model

In the first video we discussed a few key concepts that are necessary for mathematical optimization:
- parameters
- decision variables
- constraints
- objective function

In this first modeling example we will see how these are used to formulate a decision problem as an optimization model and code the formulation using `gurobipy`. For more information on all of the commands in the Python API check out our [documentation](https://www.gurobi.com/documentation/10.0/refman/py_python_api_details.html).

## The Decision Problem
Dorian Auto is considering manufacturing three types of autos: compact, midsize, and large. The resources required for, and the profits yielded by, each type of car are shown in Table 8. Currently, 6,000 tons of steel and 60,000 hours of labor are available. For production of a type of car to be economically feasible, at least 1,000 cars of that type must 
be produced. Formulate an IP to maximize Dorian’s profit.

## Sets and Define Model
Our sets are:
- $C = \{\texttt{'compact', 'midsize', 'large'}\} \quad\quad\quad\quad\quad\quad\quad\space\space \texttt{cars}$

To index each set, we'll use the lowercase letter of each set. Letters used for sets and indices are up to you. Typically, capital letters are for sets and corresponding lowercase will be the index. Single letters are used mainly for conciseness.

In [2]:
# First, import packages
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Define a gurobipy model for the decision problem
m = gp.Model('cars')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-29


## Decision Variables
This is what the optimization solver determines, which are the actions you have control over. As a reminder, they come in three main flavors:
- `Continuous`: Price of a product
- `Integer`: The number of food trucks to use for an event
- `Binary`: Yes/no decision to include a certain stock in a portfolio


### Add Variables in gurobipy
`gurobipy` let's you add decision variables primarily with two (similar) commands:
- [addVar()](https://www.gurobi.com/documentation/10.0/refman/py_model_addvar.html) adds a single variable
- [addVars()](https://www.gurobi.com/documentation/10.0/refman/py_model_addvar.html) adds a group of variables by sets/indices

When using `addVars` you have to provide the indices of the variables you want to add, which for us are the production and distribution locations. There are other arguments we can use and will cover a couple of them later on.  

### Our Decision Variables
As is often the case in writing code, there are several ways to get to the same point. Below we can see three different ways to create the decision variables.

In [3]:
# Create integer variables
x1 = m.addVar(vtype=GRB.INTEGER, name="x1")
x2 = m.addVar(vtype=GRB.INTEGER, name="x2")
x3= m.addVar(vtype=GRB.INTEGER, name="x3")

# Create binary variables
y1 = m.addVar(vtype=GRB.BINARY, name="y1")
y2 = m.addVar(vtype=GRB.BINARY, name="y2")
y3 = m.addVar(vtype=GRB.BINARY, name="y3")

m.update()

In [4]:
# Set objective
m.setObjective(2 * x1 +  3 * x2 + 4 * x3, GRB.MAXIMIZE)

In [5]:
# Define Big M
M = 5000

In [6]:
# Either or Constraint
# Constraint 1 --- x1 <= 0 OR x1 >= 1,000

c1 = m.addConstr(x1 <= M * y1, "c1")                   # either constraint

c11 = m.addConstr(1000 - x1 <= M *(1-y1), "c11")       # or constraint

In [7]:
# Either or Constraint
# Constraint 2 --- x2 <= 0 OR x2 >= 1,000 
c2 = m.addConstr(x2 <= M * y2, "c2")

c22 = m.addConstr(1000 - x2 <= M * (1-y2), "c22")

In [10]:
# Either or Constraint
# Constraint 3 --- x3 <= 0 OR x3 >= 1,000
c3 = m.addConstr(x3 <= M * y3, "c3")

c33 = m.addConstr(1000 - x3 <= M * (1-y3), "c33")

In [11]:
# Constraint 4 --- The cars produced can use at most 6,000 tons of steel.
c4 = m.addConstr(1.5 * x1 + 3 * x2 + 5 * x3 <= 6000, "c4")

In [12]:
# Constraint 5 --- The cars produced can use at most 60,000 hours of labor.
c5 = m.addConstr(30 * x1 + 25 * x2 + 40 * x3 <= 60000, "c5")
m.update()

In [13]:
# Optimize model
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Linux Mint 21.3")

CPU model: AMD Ryzen 5 5600G with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 6 columns and 21 nonzeros
Model fingerprint: 0x127c77b4
Variable types: 0 continuous, 6 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [2e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+03, 6e+04]
Found heuristic solution: objective -0.0000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 6 columns, 18 nonzeros
Variable types: 0 continuous, 6 integer (3 binary)

Root relaxation: objective 6.285714e+03, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6285.71429

In [14]:
# Print results
for v in m.getVars():
    print(f"{v.VarName} {v.X:g}")

print(f"Obj: {m.ObjVal:g}")

x1 0
x2 2000
x3 0
y1 0
y2 1
y3 0
Obj: 6000


### ----------------------------